In [40]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import getpass, os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage


JOB_DESCRIPTION = "System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie."


In [41]:

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

if not os.environ.get("SERPAPI_API_KEY"):
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
)

In [42]:
# 1. Zapytanie użytkownika

THINGS_TO_CONSIDER = [
  "1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?",
  "2. W jakim formacie chcesz otrzymać odpowiedź?",
  "3. Jaki okres czasu chcesz uwzględnić?",
]

USER_INPUT = "U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów."
prompt = (
  SystemMessage(content=JOB_DESCRIPTION) +
  SystemMessage(content=f"Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: {THINGS_TO_CONSIDER}") +
  HumanMessage(content=USER_INPUT) +
  AIMessage(content='Na czym Ci najbardziej zależy?')

).format_messages()

response1 = llm.invoke(prompt)
prompt.append(response1)
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [4]:
prompt.append(HumanMessage(content='Tytuł, czas trwania, gatunek. Chciałbym, aby formatem był JSON, lubie ten format. Zaproponuj strukturę'))
response = llm.invoke(prompt)
prompt.append(response)
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [43]:
URL = "https://helios.pl/wroclaw/kino-helios-aleja-bielany/repertuar"

from typing import List
from langchain.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from langchain_core.output_parsers import JsonOutputParser
import json
import serpapi

@tool
def scrape_raw_data_from_web() -> str:
  """
  This tool scrapes data from the web and returns it as an unstructured string
  """
  print('Scraping data from the web')
  loader = WebBaseLoader(URL)
  loader = loader.load()
  return loader[0].page_content # 'pozyskano surowy tekst' +

@tool
def select_data(raw_data_from_web: str, format='json') -> str:
  """
  This tool selects data, and extracts the fields required by the user. Often is used after scraping data from the web.
  """
  prompt_template = """
  For the following data:
  {data}

  Please extract the following fields:
  {fields}
    
  Return results in a {format} format.
  """
  fields  = ["title", "duration", "genre"]
  fields_str = "\n".join([f"- {field}" for field in fields]) # , fields: List[str],
  template = PromptTemplate(
    template=prompt_template,
    input_variables=['data', 'fields']
  )
  chain = template | llm | JsonOutputParser()
  res = chain.invoke({"data": raw_data_from_web, 'fields':fields_str, 'format':format})
  print(res)
  return json.dumps(res)

user_specification_dialogue = prompt.copy()

@tool
def try_match_specification(data: str)-> str:
  """
  This tool tries to find shortcomings within data. It strictly checks all requirements. If it fails, it will tell what's missing. Useful if no other tools are helpful.
  """
  conversation = "\n".join([p.content for p in user_specification_dialogue]) # type: ignore
  prompt_template = """
  Verify accordance of data with specification. Find shortcomings within data. Strictly checks all requirements. If data doesn't fully match the requirements, tell what's missing.
  Below is data:
  {data}

  Requirements are in following conversation:
  {conversation}    
  Provide the result as a JSON or an Error message.
  """
  prompt = prompt_template.format(data=data, conversation=conversation)
  res = llm.invoke(prompt).content
  return res

@tool
def merge_data_subsets(data_subset1: list, data_subset2: list) -> str:
  """
  This tool merges two data subsets into one. Useful after scraping or searching data from multiple sources.
  """
  prompt_template = """
    Merge two data subsets into one. Find common fields and merge them.
    Below are the data subsets:
    {data_subset1}

    {data_subset2}
  """
  template = PromptTemplate(
    template=prompt_template,
    input_variables=['data_subset1', 'data_subset2']
  )
  chain = template | llm | JsonOutputParser()
  res = chain.invoke({'data_subset1': data_subset1, 'data_subset2': data_subset2})  
  return json.dumps(res)

@tool
def find_in_google(query_template: str, elements: list, hint: str) -> str:
  """
  This tool searches for the query in google search. Then uses the hint to extract the data from the search results.
  @param query_template: a template prompt for a single query to search in google. If data contains multiple elements, then the query will be repeated for each element. E.g. We look for  of  key and movie_name is what we know. Then template may be '{movie_name} imdb rating' "
  @param data: a json containing the data to search for.
  @param hint: a hint to extract the data from the search results.
  """
  prompt_template = """
  In following search results:
  {search_results}

  Find data that matches the hint:
  {hint}    

  Return the results as a JSON.
  """
  responses = []
  for d in elements:
    query = query_template.format(**d)
    search = serpapi.search({"q": query}, api_key=os.environ["SERPAPI_API_KEY"])
    # search_results = search.get_dict()
    template = PromptTemplate(
      template=prompt_template,
      input_variables=['hint', 'search_results']
    )
    chain = template | llm | JsonOutputParser()
    res = chain.invoke({'hint': hint, 'search_results': search})
    responses.append(res)
  
  print(responses)
  return json.dumps(responses)

@tool
def think(context: str, problem_to_solve, available_tools: List[str]) -> str:
  """
  This tool is used to think about the problem and come up with a solution
  """
  prompt_template = """
  Think about the problem
  {problem_to_solve} 

  Available tools are:
  {available_tools}

   come up with a solution. Below is the context:
  {context}
  """
  prompt = prompt_template.format(problem_to_solve=problem_to_solve, available_tools=available_tools, context=context)
  res = llm.invoke(prompt)
  return res.content


tools = [ scrape_raw_data_from_web, select_data, try_match_specification, find_in_google  ]

llm_with_tools = llm.bind_tools(tools)

def run_tool(tool):
  try:
    return eval(tool['name']).invoke(tool)
  except Exception as e:
    print(f'Dyntka {e}')

counter = 0
receipe = []


prompt.append(SystemMessage(content='Wybierz aktualny krok, który należy wykonać, aby stworzyć API. W każdym kroku wykorzystaj dokładnie jedno narzędzie.'))

while True:
  counter += 1

  prompt.append(SystemMessage(content='Wybierz jedno z dostępnych narzędzi (select a too1)'))

  response = llm_with_tools.invoke("\n".join([p.content for p in prompt])) # type: ignore
  print(response)
  tool = response.tool_calls[0] # type: ignore
  if 'tool_calls' in response.additional_kwargs:
    # print(response.tool_calls)
    tool = response.tool_calls[0] # type: ignore
    try:
      tool_out = run_tool(tool)
    except Exception as e:
      print(f'Dyntka {e}')
      
    receipe.append(tool)
  prompt.append(AIMessage(content=f'Wykonałem krok: {tool["name"]} z rezultatem {tool_out}'))


  if counter > 7:
    break

prompt


content='' additional_kwargs={'tool_calls': [{'id': 'call_hwbvMkJYLiZpghpg9ptILa4I', 'function': {'arguments': '{}', 'name': 'scrape_raw_data_from_web'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 665, 'total_tokens': 679, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-5227dc08-2d9d-422e-a3be-7cea32af942b-0' tool_calls=[{'name': 'scrape_raw_data_from_web', 'args': {}, 'id': 'call_hwbvMkJYLiZpghpg9ptILa4I', 'type': 'tool_call'}] usage_metadata={'input_tokens': 665, 'output_tokens': 14, 'total_tokens': 679}
Scraping data from the web
content='' additional_kwargs={'tool_calls': [{'id': 'call_M7msTM6OJ3FksIoXkP2K7LO7', 'function': {'arguments': '{"raw_data_from_web":"\\n\\n\\nRepertuar: Kino Wrocław Helios Aleja Bielany | Helios\\n\\n\\n    Wrocław -\\n          Helios Aleja B

In [185]:
prompt

[SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_meta

In [39]:
prompt = [
 SystemMessage(content='System tworzy dla użytkownika REST API, które jednorazowo zwraca pożądane przez niego dane (rozwiązanie) w wybranym przez niego formacie.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Dla zapytania uzytkownika zadaj pytania konieczne do uzyskania deterministycznego i pożądanego rozwiązania. Poniżej masz przykładowe pytania: ['1, Na czym Ci najbardziej zależy? Na jakich informacjach, polach, cechach?', '2. W jakim formacie chcesz otrzymać odpowiedź?', '3. Jaki okres czasu chcesz uwzględnić?']", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='U: Chcę stworzyć narzędzie pozwalające na znajdywanie aktualnie granych filmów w kinach Helios. Chciałbym znać ocenę tych filmów na serwisach takich jak IMDb i Rotten Tomatoes. Chciałbym również mieć prosty dostęp do trailerów wyświetlanych filmów.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Na czym Ci najbardziej zależy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='1. Na jakich informacjach, polach, cechach najbardziej Ci zależy? (np. tytuł filmu, ocena IMDb, ocena Rotten Tomatoes, link do trailera itp.)\n2. W jakim formacie chcesz otrzymać odpowiedź? (np. JSON, XML, CSV)\n3. Jaki okres czasu chcesz uwzględnić? (np. tylko aktualne filmy, filmy z ostatniego miesiąca itp.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 235, 'total_tokens': 337, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-146fdc11-6db8-4790-9948-96ce762c8372-0', usage_metadata={'input_tokens': 235, 'output_tokens': 102, 'total_tokens': 337}),
 HumanMessage(content='Tytuł, czas trwania, gatunek. Chciałbym, aby formatem był JSON, lubie ten format. Zaproponuj strukturę', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Oto proponowana struktura JSON dla Twojego narzędzia do znajdowania aktualnie granych filmów w kinach Helios:\n\n```json\n{\n  "filmy": [\n    {\n      "tytul": "Nazwa filmu",\n      "czas_trwania": "Czas trwania w minutach",\n      "gatunek": "Gatunek filmu",\n      "ocena_imdb": "Ocena na IMDb",\n      "ocena_rotten_tomatoes": "Ocena na Rotten Tomatoes",\n      "link_do_trailera": "URL do trailera"\n    },\n    {\n      "tytul": "Nazwa drugiego filmu",\n      "czas_trwania": "Czas trwania w minutach",\n      "gatunek": "Gatunek drugiego filmu",\n      "ocena_imdb": "Ocena na IMDb",\n      "ocena_rotten_tomatoes": "Ocena na Rotten Tomatoes",\n      "link_do_trailera": "URL do trailera"\n    }\n  ]\n}\n```\n\nCzy taka struktura odpowiada Twoim potrzebom? Czy chciałbyś dodać lub zmienić jakieś pola?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 246, 'prompt_tokens': 379, 'total_tokens': 625, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-24a5c8c4-705c-4c2a-aaec-37f0d0300bcf-0', usage_metadata={'input_tokens': 379, 'output_tokens': 246, 'total_tokens': 625}),
 HumanMessage(content='Jest dobra, Wszystko cacy')
#  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YaVC0H1431UFUGKfQ7bCICw4', 'function': {'arguments': '{}', 'name': 'scrape_data_from_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 66, 'total_tokens': 79, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cb0af621-345d-4250-8eb7-100344210a28-0', tool_calls=[{'name': 'scrape_data_from_web', 'args': {}, 'id': 'call_YaVC0H1431UFUGKfQ7bCICw4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 66, 'output_tokens': 13, 'total_tokens': 79}),
]

{'messages': ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChu